In [4]:
!pip install langchain_openai langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00


In [11]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [22]:
import os
import pandas as pd
import json
from tqdm import tqdm
from typing import List
from pydantic import BaseModel
from datasets import load_dataset
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_experimental.tabular_synthetic_data.base import SyntheticDataGenerator
from huggingface_hub import notebook_login

In [38]:
# Make sure you set your OpenAI API key as an environment variable
# or replace os.getenv with your key (not recommended for security reasons)
#openai_api_key = os.getenv("OPENAI_API_KEY")
#openai_api_key = ''

In [7]:
class Message(BaseModel):
    role: str
    content: str

In [8]:
class ChatMLConversation(BaseModel):
    messages: List[Message]

In [9]:
def format_response_messages(messages):
    def escape_braces(text):
        return text.replace("{", "{{").replace("}", "}}")
    return "\n".join(
        f'{msg["role"].capitalize()}: {escape_braces(msg["content"])}'
        for msg in messages
    )

In [33]:
notebook_login()

In [21]:
dataset = load_dataset("capstone-research/bitext-cleaned-travel-dataset-3k")

df = dataset["train"].to_pandas()

print(df.head())
print(f"Total rows: {len(df)}")

   Unnamed: 0                                        instruction  \
0           0  id like information about my checked carry-on ...   
1           1  I watn to see the  checked carry-on baggage al...   
2           2  id like to know about my checked baggage allow...   
3           3  i want information about my checked baggage al...   
4           4  igot to check the  checked carry-on baggage al...   

                    intent                 function  \
0  check_baggage_allowance  check_baggage_allowance   
1  check_baggage_allowance  check_baggage_allowance   
2  check_baggage_allowance  check_baggage_allowance   
3  check_baggage_allowance  check_baggage_allowance   
4  check_baggage_allowance  check_baggage_allowance   

                                            response  
0  Okay, I can help you figure out your checked l...  
1  Okay, I can definitely help you with that. To ...  
2  Okay, I can definitely help you with that! To ...  
3  Okay, I can help you with that!  To f

In [23]:
# Template for examples
example_template = """
Instruction: {instruction}
Intent: {intent}
Function: {function}
Reference Response: {response}
"""

In [24]:
example_prompt = PromptTemplate(
    input_variables=["instruction", "intent", "function", "response"],
    template=example_template
)

In [25]:
# Prefix and suffix for the prompt
prefix = """You are a helpful assistant that produces multi-turn conversations in ChatML + ReAct + JSON toolstyle format for function calling for customer support queries in the travel domain.

Use the examples below to learn the format.
"""

In [26]:
suffix = """
Enhance the provided few-shot examples and generate a realistic, multi-turn conversation in ChatML + ReAct + JSON toolstyle format for the user query below.

Context:
Instruction: {instruction}
Intent: {intent}
Function: {function}
Reference Response: {response}

Your Task:
1. Begin with a single system message that sets context or welcomes the user (e.g., "Welcome to Travel Support. How can I assist you today?"). Only include one system message at the beginning of the conversation.
2. Simulate a natural user message based on the instruction, annotated with [intent=...] and [sentiment=...]. Sentiment should be classified as one of: positive, neutral, or negative.
3. For assistant responses, follow the ReAct pattern:
   - **Thought:** Internal reasoning about the user's request or current step.
   - **Action:** A decision to take a specific action such as calling a tool or querying information
   - **FunctionCall:** Once the required input is provided by the user, simulate a tool function call like: {{ "function": "<function_name>", "parameters": {{ "key": "value" }} }}. Populate this only when all parameters needed for the function are available and embed the response reasoning inside the assistant's Response field.
   - **Observation:** The result or feedback from the action taken
   - **Response:** Provide a natural response to the user. Simulate tool responses inline where appropriate.
4. If additional info (like a PNR or Ticket Number) is required, the assistant should first request it without invoking the function yet.
5. Ensure the structure follows the ChatML-style JSON format exactly:
{{
  "messages": [
    {{ "role": "system", "content": "Welcome to Travel Support. How can I assist you today?" }},
    {{ "role": "user", "content": "[intent={intent}][sentiment=...] <natural user message>" }},
    {{ "role": "assistant", "content": "Thought: ...\\nAction: ...\\nAction Input: ...\\nResponse: ..." }},
    ...
  ]
}}

Return only the valid JSON object without any markdown or explanation.
"""

In [27]:
# Initialize LLM using OpenAI
llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0.7,
    openai_api_key=openai_api_key
)

In [31]:
!mkdir -p /content/synthetic_dataset

In [34]:
!git clone https://huggingface.co/datasets/capstone-research/travel_multi_turn_chat_fewshot_examples

Cloning into 'travel_multi_turn_chat_fewshot_examples'...
remote: Enumerating objects: 38, done.
remote: Total 38 (delta 0), reused 0 (delta 0), pack-reused 38 (from 1)
Unpacking objects: 100% (38/38), 52.86 KiB | 2.78 MiB/s, done.


In [35]:
!mv travel_multi_turn_chat_fewshot_examples fewshot_examples

In [29]:
# Output path
output_path = "/content/synthetic_dataset"

In [36]:
# Generate synthetic data
for i in tqdm(range(len(df))):
    current_row = df.iloc[i]

    try:
        with open(f'fewshot_examples/{current_row["intent"]}_fewshot.json', 'r') as file:
            fewshot_examples = json.load(file)

        for example in fewshot_examples:
            res = json.loads(example["response"])
            if isinstance(res, (list, dict)):
                example["response"] = format_response_messages(res)
            elif not isinstance(example["response"], str):
                example["response"] = str(example["response"])

        # Construct few-shot prompt
        few_shot_prompt = FewShotPromptTemplate(
            examples=fewshot_examples,
            example_prompt=example_prompt,
            prefix=prefix,
            suffix=suffix,
            input_variables=["instruction", "intent", "function", "response"]
        )

        generator = SyntheticDataGenerator(
            template=few_shot_prompt,
            llm=llm,
            output_schema=ChatMLConversation
        )

        conversations = generator.generate(
            subject=current_row["intent"],
            runs=1,
            extra="Focus on customer support for the travel domain with different personas and utterances",
            instruction=current_row["instruction"],
            intent=current_row["intent"],
            response=current_row["response"],
            function=current_row["function"]
        )

        with open(f"{output_path}/{current_row['intent']}.jsonl", "a", encoding="utf-8") as f:
            for convo in conversations:
                f.write(json.dumps(json.loads(convo.strip()), ensure_ascii=False) + "\n")

    except Exception as e:
        print(f"Error generating for row {i}: {e}")

print(f"Dataset saved to {output_path}")

  0%|          | 1/3300 [00:13<12:16:56, 13.40s/it]


KeyboardInterrupt: 

**Do not forget to download the data after generating synthetic data**

In [37]:
!tar -cvzf synthetic_dataset.tar.gz synthetic_dataset/*

synthetic_dataset/check_baggage_allowance.jsonl
